# 1) Données

In [1]:
import pandas as pd
import numpy as np
import csv
import os

# calcul de la position dans le roulement d'un poste en fonction de la semaine
def rFunction(s,p):
    return ((p-s)%24)

# calcul du numéro de semaine correspondant au jour j
def sFunction(j):
    return j//7

# calcul du jour non travaillé pour une semaine donnée d'un opérateur à 80%
# en fonction du jour non travaillé la première semaine
def delta_i(s,d_i):
    return (d_i+s)%5

# operateurs + competences
dataOperateurs = pd.read_csv('./data/operateurs.csv',sep = ";" )

cardO = len(dataOperateurs.axes[0])
colsDataOperateurs = len(dataOperateurs.axes[1])
nomsOperateurs = dataOperateurs.iloc[0:cardO,0:3]
competencesOperateurs = dataOperateurs.iloc[0:cardO,3:(colsDataOperateurs+1)]

# postes (intitulés + rho)
dataPostes = pd.read_csv('./data/postes.csv',sep = ";" )
cardP = len(dataPostes.axes[0])
nomsPostes = dataPostes.iloc[0:cardO,1]
rho = [0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1]

# le nombre de semaines <=> nombre de postes <=> nombre d'opérateurs
#cardS = cardO
cardS = 2
cardJ = 7*cardS
cardR = cardO

# kappa_ip (opérateur possède la compétence pour le poste)
#kappa = pd.read_csv('./data/kappa_ip.csv',sep = ";" )
kappa = [
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0]]

# sigma_pp' (postes sur le même créneau horaire)
#sigma = pd.read_csv('./data/sigma_pp.csv',sep = ";" )
sigma= [[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1]]

d = [3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]

# o_j (jour ouvrés)
o = np.zeros(cardJ)
for j in range(0,len(o)):
    if (j%7 <= 4):
        o[j] = 1

# H_s (jours de chaque semaine)
H = []
for i in range(0,cardS):
    H.append(np.zeros(7))
for i in range(0,cardS):
    for j in range(0,7):
        H[i][j]=(i*7 + j)
        
profils = np.zeros(cardO)
cardProfils = 0
for i in range(0,cardO):
    found = False
    for j in range(0,i):
        if not found:
            diff = False
            if d[i] == d[j]:
                for p in range(0,cardP):
                    if kappa[i][p] != kappa[j][p]:
                        diff = True
            else:
                diff = True
        
            if not diff:
                profil = profils[j]
                found = True
    if found:
        profils[i] = profil
    else:
        profils[i] = cardProfils
        cardProfils = cardProfils + 1
        
        
print(profils)
print(cardProfils)
N = np.zeros(cardProfils)
for i in range(cardO):
    N[int(profils[i])] += 1
print(N)

# delta_ij (est-ce qu'une personne à 80% travaille le jour j)
#delta = pd.read_csv('./data/delta_ij.csv',sep = ";" )
# d[i] représente le jour non travaillé de l'opérateur i la première semaine, d[i]=-1 si i est à temps plein

delta = np.full((cardProfils, cardJ), 1)
for profil in range(cardProfils):
    found = False
    i = 0
    index = -1
    while not found and i < cardO:
        if profils[i] == profil:
            found = True
            index = i
        i += 1
    if d[index] != -1:
        d_i = d[index]
        for s in range(0, cardS):
            d_is = delta_i(s,d_i) #rang du jour non travaillé la semaine s
            j_s = 7*s + d_is      #numéro du jour non travaillé
            delta[profil][j_s] = 0
            
kappaProfils = []
for profil in range(cardProfils):
    found = False
    i = 0
    index = -1
    while not found and i < cardO:
        if profils[i] == profil:
            found = True
            index = i
        i += 1
    kappaProfils.append(kappa[index])
    
print(kappaProfils)
    
    

print(delta[0])

#print(nomsOperateurs)
#print(competencesOperateurs)
#print(nomsPostes)
#print(rho)
#print(kappa)
#print(sigma)
#print(delta)
#print(o)
#print(H[1])


[0. 1. 2. 1. 1. 2. 3. 1. 2. 1. 4. 1. 1. 1. 1. 1. 2. 3. 1. 3. 2. 1. 1. 3.]
5
[ 1. 13.  5.  4.  1.]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]]
[1 1 1 0 1 1 1 1 1 1 1 0 1 1]


# 2) Modèle mathématique


## 2.1) Variables

In [2]:
from pulp import *

# x_ijp = 1 si l'opérateur prend en charge le poste p le jour j
x = {} 
for prof in range(0,cardProfils):
    for j in range(0,cardJ): 
        for p in range(0,cardP): 
            x[prof,j,p]=LpVariable(f"x({prof,j,p})", cat=LpBinary) #LpBinary

# y_ir = 1 si l'opérateur i est position r dans le roulement
y = {} 
for prof in range(0,cardProfils):
    for r in range(0,cardR):    
        y[prof,r]=LpVariable(f"y({prof,r})", cat=LpBinary)
        
# z_isp = 1 si l'opérateur est affecté au poste p la semaine s
z = {} 
for prof in range(0,cardProfils):
    for s in range(0,cardS): 
        for p in range(0,cardP): 
            z[prof,s,p]=LpVariable(f"z({prof,s,p})", cat=LpBinary)

# u_is = 1 si l'opérateur i est affecté à un poste différent de celui prévu la semaine s
u = {} 
for p in range(0,cardP):
    for s in range(0,cardS):    
        u[p,s]=LpVariable(f"u({p,s})", cat=LpBinary)

print("nb de variables x_ijp = ", len(x))
print("nb de variables y_ir  = ", len(y))
print("nb de variables z_isp = ", len(z))
print("nb de variables u_ir  = ", len(u))
print()
print("nb total de variables = ", len(x)+len(y)+len(z)+len(u))

nb de variables x_ijp =  1680
nb de variables y_ir  =  120
nb de variables z_isp =  240
nb de variables u_ir  =  48

nb total de variables =  2088


## 2.2) Création du modèle avec PuLP

In [3]:
prob = LpProblem("ModelPlanning", LpMinimize)

prob += sum(u[p,s] for p in range(0,cardP) for s in range(0,cardS)), "obj"

## 2.2) Contraintes

In [4]:
#contrainte 1 Un opérateur ne peut prendre un poste que s'il est compétent et disponible sur un jour ouvré

for prof in range(0,cardProfils):
    for j in range(0,cardJ):
        for p in range(0,cardP):
            prob += (x[prof,j,p] <= delta[prof][j]*kappaProfils[prof][p]*o[j], f"C1_{prof,j,p}")

#contrainte 2 Un opérateur ne peut être affecté à un poste une semaine que s'il en possède la compétence

for prof in range(0,cardProfils):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += (z[prof,s,p] <= kappaProfils[prof][p], f"C2_{prof,s,p}")

#contrainte 3 Chaque jour, l'opérateur est affecté à son poste de la semaine (s'il n'est pas rouleur)
for prof in range(0,cardProfils):
    for s in range(0,cardS):
        for j in H[s]:
            for p in range(0,cardP):
                prob += x[prof,j,p] <= z[prof,s,p] + sum((1 - rho[p2])*z[prof,s,p2] for p2 in range(0,cardP)), f"C3_{prof,j,p}"

#contrainte 4 (page 8) Chaque jour, un opérateur est affecté à au plus un poste #MODIF

for prof in range(0,cardProfils):
    for j in range(0,cardJ):
        if o[j] == 1 and delta[prof][j] == 1:
            prob += sum(x[prof,j,p] for p in range(0,cardP)) == N[prof], f"C4_{prof,j,p}"

#contrainte 5 (page 8) Chaque jour, chaque poste est assuré par un opérateur #MODIF
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 1:
            prob += sum(x[prof,j,p] for prof in range(0,cardProfils)) == 1, f"C5_{prof,j,p}"
            
#contrainte 5.2 BONUS Chaque jour, un poste roulant est occupé par au plus une personne
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 0:
            prob += sum(x[prof,j,p] for prof in range(0,cardProfils)) <= 1, f"C52_{prof,j,p}"

#contrainte 6 (page 9) Chaque semaine, chaque opérateur est affecté à un poste

for prof in range(0,cardProfils):
    for s in range(0,cardS):
        prob += sum(z[prof,s,p] for p in range(0,cardP)) == N[prof], f"C6_{prof,s,p}"

#contrainte 7 (page 9) Chaque semaine, chaque poste est assuré par un opérateur

for p in range(0,cardP):
    for s in range(0,cardS):
        prob += sum(z[prof,s,p] for prof in range(0,cardProfils)) == 1, f"C7_{prof,s,p}"

#contrainte 8 (page 10) Chaque opérateur est affecté à une position dans le roulement

for prof in range(0,cardProfils):
    prob += sum(y[prof,r] for r in range(0,cardR)) == N[prof], f"C8_{prof,r}"

#contrainte 9 (page 10) Chaque position dans le roulement est associée à un opérateur

for r in range(0,cardR):
    prob += sum(y[prof,r] for prof in range(0,cardProfils)) == 1, f"C9_{prof,r}"

#contrainte 10 (page 11) Un opérateur peut changer de poste si l'horaire de ce dernier est compatible

for prof in range(0,cardProfils):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += z[prof,s,p] <= sum(sigma[p][p2]*y[prof,rFunction(s,p2)] for p2 in range(0,cardP)), f"C10_{prof,s,p}"

#contrainte 11 (page 11) Lorsqu'un jour un opérateur est affecté à un poste A alors qu'il est affecté à un poste B pour cette semaine, cela augmente son score d'insatisfaction
for prof in range(0,cardProfils):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += u[p,s] >= (kappaProfils[prof][p]*rho[p]*y[prof,rFunction(s,p)] - z[prof,s,p]), f"C11_{prof,p,s}"


## 2.4) Résolution du problème

In [5]:
import time

prob.writeLP("Model.lp")

start = time.time() 
prob.solve(CPLEX_PY())
end = time.time()

duree = end - start
print("Temps de résolution : " + str(duree))

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 2662 rows and 672 columns.
MIP Presolve modified 288 coefficients.
Aggregator did 2 substitutions.
Reduced MIP has 1791 rows, 1414 columns, and 12672 nonzeros.
Reduced MIP has 1414 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (10.13 ticks)
Found incumbent of value 36.000000 after 0.03 sec. (19.86 ticks)
Probing changed sense of 60 constraints.
Probing time = 0.01 sec. (3.01 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 72 rows and 60 columns.
Reduced MIP has 1719 rows, 1354 columns, and 12492 nonzeros.
Reduced MIP has 1354 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (13.68 ticks)
Probing time = 0.00 sec. (2.54 ticks)
Clique table members: 7050.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode:

## 2.5) Extraction de la solution

In [6]:
print("objectif =", value(prob.objective))

objectif = 0.0


## 2.6) Affichage de la solution

In [7]:
print("uis")
for s in range(0,cardS):
    for p in range(0,cardP):
        if(u[p,s].varValue != 0):
            print("u[",i,",",s,"] =", u[p,s].varValue)

print('Affectation aux roulements')
for i in range(cardProfils):
    for r in range(0,24):
        if(y[i,r].varValue != 0):
            print('y[',i,',',r,'] =', y[i,r].varValue)

print()
print('Affectations hebdomadaires')
for s in range(0,cardS):
    print('semaine',s)
    for i in range(0,cardProfils):
        for p in range(0,cardP):
            if z[i,s,p].varValue==1:
                print('semaine',s,',',i,'affecté au poste',p)
                
print('======================================================')

for p in range(0,cardP):
    print()
    if rho[p] == 1:
        print('poste',p,'non rouleur')
    else:
        print('poste',p,'rouleur')
    for j in range(0,cardJ):
        for i in range(0,cardProfils):
            if (x[i,j,p].varValue == 1):
                print(i,'occupe le poste',p,'le jour',j)
                
print('======================================================')

                
for i in range(0,cardO):
    print()
    print('planning de l\'opérateur',i)
    for j in range(0,cardJ):
        for p in range(0,cardP):
            if (x[i,j,p].varValue == 1):
                print('poste',p,'le jour',j)

uis
Affectation aux roulements
y[ 0 , 22 ] = 1.0
y[ 1 , 0 ] = 1.0
y[ 1 , 2 ] = 1.0
y[ 1 , 4 ] = 1.0
y[ 1 , 5 ] = 1.0
y[ 1 , 6 ] = 1.0
y[ 1 , 9 ] = 1.0
y[ 1 , 10 ] = 1.0
y[ 1 , 13 ] = 1.0
y[ 1 , 14 ] = 1.0
y[ 1 , 15 ] = 1.0
y[ 1 , 16 ] = 1.0
y[ 1 , 17 ] = 1.0
y[ 1 , 19 ] = 1.0
y[ 2 , 1 ] = 1.0
y[ 2 , 11 ] = 1.0
y[ 2 , 12 ] = 1.0
y[ 2 , 20 ] = 1.0
y[ 2 , 21 ] = 1.0
y[ 3 , 3 ] = 1.0
y[ 3 , 7 ] = 1.0
y[ 3 , 8 ] = 1.0
y[ 3 , 18 ] = 1.0
y[ 4 , 23 ] = 1.0

Affectations hebdomadaires
semaine 0
semaine 0 , 0 affecté au poste 22
semaine 0 , 1 affecté au poste 0
semaine 0 , 1 affecté au poste 2
semaine 0 , 1 affecté au poste 4
semaine 0 , 1 affecté au poste 5
semaine 0 , 1 affecté au poste 6
semaine 0 , 1 affecté au poste 9
semaine 0 , 1 affecté au poste 10
semaine 0 , 1 affecté au poste 13
semaine 0 , 1 affecté au poste 14
semaine 0 , 1 affecté au poste 15
semaine 0 , 1 affecté au poste 16
semaine 0 , 1 affecté au poste 17
semaine 0 , 1 affecté au poste 19
semaine 0 , 2 affecté au poste 1
semain

poste 16 le jour 1
poste 17 le jour 1
poste 18 le jour 1
poste 1 le jour 2
poste 2 le jour 2
poste 9 le jour 2
poste 10 le jour 2
poste 11 le jour 2
poste 12 le jour 2
poste 13 le jour 2
poste 14 le jour 2
poste 15 le jour 2
poste 17 le jour 2
poste 18 le jour 2
poste 19 le jour 2
poste 20 le jour 2
poste 3 le jour 3
poste 6 le jour 3
poste 7 le jour 3
poste 8 le jour 3
poste 9 le jour 3
poste 10 le jour 3
poste 11 le jour 3
poste 13 le jour 3
poste 14 le jour 3
poste 15 le jour 3
poste 17 le jour 3
poste 21 le jour 3
poste 22 le jour 3
poste 1 le jour 4
poste 4 le jour 4
poste 5 le jour 4
poste 9 le jour 4
poste 10 le jour 4
poste 11 le jour 4
poste 12 le jour 4
poste 13 le jour 4
poste 14 le jour 4
poste 15 le jour 4
poste 17 le jour 4
poste 19 le jour 4
poste 20 le jour 4
poste 0 le jour 7
poste 1 le jour 7
poste 8 le jour 7
poste 9 le jour 7
poste 10 le jour 7
poste 11 le jour 7
poste 12 le jour 7
poste 14 le jour 7
poste 15 le jour 7
poste 16 le jour 7
poste 17 le jour 7
poste 20 

KeyError: (5, 0, 0)

# Extraction des données au format CSV

In [ ]:
import csv
with open('res.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    jours = [""]
    for s in range(0,cardS):
        jours.extend(["lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"])
    writer.writerow(jours)
    
    dates = [""]
    for j in range(0,cardJ):
        dates.append("jour "+str(j))
    writer.writerow(dates)
    
    for i in range(0,cardO):
        planningPerso = [nomsOperateurs.iloc[i,1] + " " + nomsOperateurs.iloc[i,2]]
        for j in range(0,cardJ):
            found = False
            for p in range(0,cardP):
                if x[i,j,p].varValue == 1:
                    found = True
                    planningPerso.append(nomsPostes.iloc[p])
            if not found:
                if o[j] == 0:
                    planningPerso.append("")
                else:
                    planningPerso.append("RA")
        writer.writerow(planningPerso)